In [13]:
from numpy.polynomial import Polynomial
from numpy.polynomial.polynomial import polydiv
from phe import paillier
from numpy import prod
import secrets

In [2]:
class Player:
    """
    Class of honest-but-curious adversaries.

    Parameters
    ----------
    S : set_like
    sk : secret key to a homomorpic cryptosystem
    """
    def __init__(self, S, sk):
        self.S = S
        self.k = len(S)
        self.sk = sk
        self.f_list = []
    
    def polynomial(self):
        # cria o polinómio f a partir do conjunto S 
        self.f = Polynomial.fromroots(self.S)
    
    def encrypt(self, pk):
        # encipta o polinómio f com pk
        self.ef = Polynomial([pk.encrypt(x) for x in self.f.coef])
    
    def append_f(self, encrypted_polynomial):
        # adiciona polinomio encriptado de outro jogador a f_list
        self.f_list.append(encrypted_polynomial)

    def receive_l(self, encrypted_polynomial):
        # recebe lambda_i-1
        self.lam_im1 = encrypted_polynomial
    
    def receive_p(self, encrypted_polynomial):
        #recebe p = lambda_n
        self.ep = encrypted_polynomial

    def phi(self):
        # calcula phi_i
        c = len(self.f_list)
        r = []
        for _ in range(c):
            r.append(Polynomial([secrets.randbelow(N) for _ in range(self.k + 1)])) #N a ser substituído pelo domínio
        phi_i = sum([prod([self.f_list[j],r[j]]) for j in range(c)]) #é suposto encriptar o phi, ou ele por construção já é encriptado?
        self.phi = Polynomial([sk.encrypt(x) for x in phi_i.coef]) #assumindo que encriptas o phi com o sk

    def lam(self):
        # calcula lambda_i
        self.lam_i = self.lam_im1 + self.phi_i
        for x in self.lam_i.coef:
            x.obfuscate() #re-randomização dos coeficientes
    
    def decrypt(self):
        self.p = [sk.decrypt(x) for x in self.ep.coef]

    def multiset(self):
        # calcula o multiset de intersecção na forma [(a,b),...,(a,b)], a \in S_i
        # onde o elemento a aparece b vezes no multiset
        p = self.p
        res = []
        for a in self.S:
            b = 0
            quo, rem = polydiv(p, (-a, 1))
            while all(rem == 0):
                b += 1
                p = quo
                quo, rem = polydiv(p, (-a, 1))
            res.append((a,b))
        self.intersection = res

    

Soma de polinómios encriptados: `a.ef + b.ef`

Produto de polinímio não encriptado com polinómio encriptado: `a.f * b.ef`

Derivada de polinómio encriptado: `a.ef.deriv(d)`

Avaliação de um polinómio encriptado num ponto não encriptado: `a.ef(x)`


Função da secção 5.1

In [ ]:
def psi(players, c, pk):
    """
    Perform a private Set-Intersection secure against a coalition of honest-but-curious
    adversaries.

    Parameters
    ----------
    players : array_like construct of class Player elements
    c : dishonestly colluding constant
    pk: public key to a homomorpic cryptosystem
    """
    n = len(players)
    assert(n >= 2, "Not enough players")
    assert(c < n, "Dishonestly colluding too large")
    assert(all([player.k == player[0].k for player in players]),\
        "Differently sized private sets")
    #1
    for player in players:
        #a
        player.polynomial()       
        player.encrypt()
    
    #b
    for i in range(n):
        for j in range(1, c+1):
            player.append_f(players[(i-j) % n].ef)
        
        # Qual é o domínio?? #consoante a resposta é preciso alterar a função phi
        # DEFAULT_KEYSIZE = 3072
        # BASE = 16
        #c, d
        players[i].phi()

    #2
    players[1].receive_l(players[0].phi_i)

    #3
    for i in range(1, n):
        #a é realizado implicitamente
        #b
        players[i].lam()

        #c
        players[(i+1)%len(players)].receive_l(players[i].lam_i)
    #4
    for player in players:
        player.receive_p(players[0].lam_i)

    #5
    for player in players:
        player.decrypt()
    #6
    for player in players:
        player.multiset()

In [6]:
# Generate keys
pk, sk = paillier.generate_paillier_keypair()

In [7]:
# Create players
a = Player([1,2,3],sk)#aqui não devia ser pk?
b = Player([2,3,4],sk)

In [ ]:
# Calculate private set
psi()

In [ ]:
#find private set

In [ ]:
#get obtained provate sets

In [ ]:
#compare make sure the sets are the same

In [31]:
# Teste das operações definidas em 4.2
pk, sk = paillier.generate_paillier_keypair()
a = Player([1,2,3],sk)
b = Player([2,3,4],sk)
a.polynomial()
b.polynomial()
a.encrypt(pk)
b.encrypt(pk)

d = 2
x = 0
print(f"Soma:\n{a.ef + b.ef}")
print(f"Multiplicação:\n{a.f * b.ef}")
print(f"Derivada:\n{a.ef.deriv(d)}")
print(f"Avaliação:\n{a.ef(x)}")


Soma:
<phe.paillier.EncryptedNumber object at 0x75d85414ca40> +
<phe.paillier.EncryptedNumber object at 0x75d84ff1c950>·x +
<phe.paillier.EncryptedNumber object at 0x75d84ff1fe90>·x² +
<phe.paillier.EncryptedNumber object at 0x75d84ff1ec60>·x³
Multiplicação:
<phe.paillier.EncryptedNumber object at 0x75d8546ffc80> +
<phe.paillier.EncryptedNumber object at 0x75d84ff1e4e0>·x +
<phe.paillier.EncryptedNumber object at 0x75d84ff1fd40>·x² +
<phe.paillier.EncryptedNumber object at 0x75d84ff1c530>·x³ +
<phe.paillier.EncryptedNumber object at 0x75d84ff1fe90>·x⁴ +
<phe.paillier.EncryptedNumber object at 0x75d84ff1c950>·x⁵ +
<phe.paillier.EncryptedNumber object at 0x75d84ff1f290>·x⁶
Derivada:
<phe.paillier.EncryptedNumber object at 0x75d8541cd400> +
<phe.paillier.EncryptedNumber object at 0x75d84ff1c530>·x
Avaliação:
